# import the required libraries used for nlp


In [5]:
import nltk
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import  PorterStemmer

In [6]:
nltk.download('stopwords')
  

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [7]:
from nltk.stem import WordNetLemmatizer

In [8]:
nltk.download("wordnet")

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

# get the file

In [9]:
the_file=pd.read_csv("spam_classifier_data/SMSSpamCollection", sep="\t", names=["labels", "message"])

In [10]:
the_file.head(6)

,labels,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...


# clean the data

In [11]:
#this is done using lammatizing
wl=WordNetLemmatizer()
list_clean_message=[]
for i in range(len(the_file)):
    decide=re.sub('[^a-zA-Z]'," ", the_file['message'][i])
    decide=decide.lower()
    decide=decide.split()
    decide=[wl.lemmatize(word ) for word in decide if not word in stopwords.words("english")]
    decide=" ".join(decide)
    list_clean_message.append(decide)
print(list_clean_message[:5], sep="\n")
#this is using stemming
# ps=PorterStemming()
# list_clean_message=[]
# for i in range(len(the_file)):
#     decide=re.sub('[^a-zA-Z]'," ", the_file['message'][i])
#     decide=decide.lower()
#     decide=decide.split()
#     decide=[ps.stem(word ) for word in decide if not word in stopwords.words("english")]
#     decide=" ".join(decide)
#     list_clean_message.append(decide)
# print(list_clean_message[:5], sep="\n")
    


['go jurong point crazy available bugis n great world la e buffet cine got amore wat', 'ok lar joking wif u oni', 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply', 'u dun say early hor u c already say', 'nah think go usf life around though']


In [12]:
print(len(list_clean_message))

5572


In [13]:
# print(list_clean_message[:5])

# Bag of words

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(max_features=5000)
X=tf.fit_transform(list_clean_message)
#this is using count vector
# from sklearn.feature_extraction.text import CountVectorizer
# cv=CountVectorizer(max_features=5000)
# X=cv.fit_transform(list_clean_message)


In [15]:
X

<5572x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 43040 stored elements in Compressed Sparse Row format>

# dummy vector and turning the labels such that its readable

In [16]:
Y=pd.get_dummies(the_file["labels"])
Y=Y.iloc[:,1]
Y.head(20)

0     0
1     0
2     1
3     0
4     0
5     1
6     0
7     0
8     1
9     1
10    0
11    1
12    1
13    0
14    0
15    1
16    0
17    0
18    0
19    1
Name: spam, dtype: uint8

In [17]:
# now we have x(independednt), y(dependent),enties for our training
from sklearn.model_selection  import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.33, random_state=0)

In [19]:
x_train.shape

(3733, 5000)

In [20]:
x_test.shape

(1839, 5000)

# classifier for prediction

In [21]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
spam_detection_model=mnb.fit(x_train, y_train)

In [22]:
final_deal=spam_detection_model.predict(x_test)

In [23]:
final_deal[:20]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
      dtype=uint8)

In [72]:
# y_test[:20] you can do thisto check whether the output is somewhat similar or not

# to check the accuracy

In [99]:
from sklearn.metrics import confusion_matrix
confusion=confusion_matrix(y_test,final_deal )

In [100]:
confusion

array([[1596,    1],
       [  45,  197]], dtype=int64)

# to check the accuracy

In [101]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(final_deal, y_test)
accuracy*100

97.49864056552474